In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [2]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:08<00:00, 1139.47it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:376: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_full.groupby("bin")


{'basic': {'expected_sample_burst_size': 3.001476466563434,
           'expected_sample_data_size': 16.494616298750405,
           'expected_time_to_recover': 99.0941,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.16146269051336054,
             'bit_redundancy': 415.4705,
             'ideal_bit_efficency': 0.19084357801182156,
             'ideal_bit_redundancy': 339.19146996418453,
             'ideal_packet_efficiency': 0.9542178900591077,
             'ideal_packet_redundancy': 67.83829399283691,
             'ideal_time_to_recover': 83.83829399283691,
             'ideal_transmitted_bits': 419.19146996418453,
             'packet_efficiency': 0.8073134525668026,
             'packet_redundancy': 83.0941,
             'permeability': 0.8460472822583475,
             'saturation': 16.0,
             'transmitted_bits': 495.4705}}
Found implementation: shuffled_binary_fullrank
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:06<00:00, 1614.29it/s]


{'basic': {'expected_sample_burst_size': 3.015243428215897,
           'expected_sample_data_size': 16.498055685176542,
           'expected_time_to_recover': 14.229299999999999,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.32683834522814414,
             'bit_redundancy': 47.893095671225836,
             'ideal_bit_efficency': 0.3865724855016302,
             'ideal_bit_redundancy': 36.89936184287851,
             'ideal_packet_efficiency': 1.9328624275081512,
             'ideal_packet_redundancy': 7.379872368575702,
             'ideal_time_to_recover': 12.030553234330533,
             'ideal_transmitted_bits': 60.152766171652665,
             'packet_efficiency': 1.6341917261407206,
             'packet_redundancy': 9.578619134245168,
             'permeability': 0.8454775171182374,
             'saturation': 4.650680865754831,
             'transmitted_bits': 71.14649999999999}}
Found implementation: simple_matrix
  F

...

{'basic': {'expected_sample_burst_size': 2.99468036737292,
           'expected_sample_data_size': 16.45298717832136,
           'expected_time_to_recover': 32.504,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.1538272212650751,
             'bit_redundancy': 137.51999999999998,
             'ideal_bit_efficency': 0.18182598857080945,
             'ideal_bit_redundancy': 112.4940963968092,
             'ideal_packet_efficiency': 0.909129942854047,
             'ideal_packet_redundancy': 22.498819279361843,
             'ideal_time_to_recover': 27.498819279361843,
             'ideal_transmitted_bits': 137.4940963968092,
             'packet_efficiency': 0.7691361063253754,
             'packet_redundancy': 27.503999999999998,
             'permeability': 0.8460133915629413,
             'saturation': 5.0,
             'transmitted_bits': 162.51999999999998}}
Found implementation: single_binary_fullrank
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3459.38it/s]


{'basic': {'expected_sample_burst_size': 3.0196281816005746,
           'expected_sample_data_size': 16.594430702944226,
           'expected_time_to_recover': 13.324999999999998,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3490192019328204,
             'bit_redundancy': 43.37159567122583,
             'ideal_bit_efficency': 0.41252895631620506,
             'ideal_bit_redundancy': 33.114528086012086,
             'ideal_packet_efficiency': 2.0626447815810254,
             'ideal_packet_redundancy': 6.6229056172024166,
             'ideal_time_to_recover': 11.273586482957247,
             'ideal_transmitted_bits': 56.36793241478624,
             'packet_efficiency': 1.7450960096641017,
             'packet_redundancy': 8.674319134245167,
             'permeability': 0.846047766075591,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.62499999999999}}
Found implementation: single_binary_ful

benchmark: 100%|██████████| 10000/10000 [00:03<00:00, 3328.39it/s]


{'basic': {'expected_sample_burst_size': 2.9955223880597015,
           'expected_sample_data_size': 16.463236449332285,
           'expected_time_to_recover': 13.794699999999999,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.33713533935169526,
             'bit_redundancy': 45.72009567122585,
             'ideal_bit_efficency': 0.3984778621261323,
             'ideal_bit_redundancy': 35.10216956610292,
             'ideal_packet_efficiency': 1.9923893106306616,
             'ideal_packet_redundancy': 7.020433913220584,
             'ideal_time_to_recover': 11.671114778975415,
             'ideal_transmitted_bits': 58.355573894877075,
             'packet_efficiency': 1.6856766967584764,
             'packet_redundancy': 9.144019134245168,
             'permeability': 0.8460578902749183,
             'saturation': 4.650680865754831,
             'transmitted_bits': 68.9735}}
